In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.init as init
import csv

In [2]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5, 0.5), (0.5, 0.5, 0.5))
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
train_datapath = './iith-dl-contest-2024/train/train'
train_data = datasets.ImageFolder(root=train_datapath, transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
# # how to access the train data
# for i, (data, target) in enumerate(train_loader):
#     print(data.shape)

In [4]:
# create a map for the lables corrosponds to the folder names
label_map = train_data.class_to_idx
print(label_map['n01443537'])

# create a map for the folder names corrosponds to the lables
label_map = dict((v,k) for k,v in label_map.items())
print(label_map[0])

0
n01443537


In [5]:
from torchvision import models

# ResNet-18 model
# model = models.resnet18(weights=None).to(device)
model=models.convnext_tiny(weights=None).to(device)

# # Initialize the weights using normal initialization
# def normal_weights_init(m):
#     if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
#         init.normal_(m.weight, mean=0, std=0.01)
#         if m.bias is not None:
#             init.constant_(m.bias, 0)

# # Apply normal initialization to the model
# model.apply(normal_weights_init)

# printing the number of parameters
print(sum(p.numel() for p in model.parameters() if p.requires_grad))


28589128


In [6]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available. GPU detected.")
else:
    print("CUDA is not available. CPU will be used.")

cuda
CUDA is available. GPU detected.


In [7]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [8]:
# calculate the test data accuracy
test_datapath = './iith-dl-contest-2024/test' # this folder itself contains images

# read all the images in test_datapath
test_data = datasets.ImageFolder(root=test_datapath, transform=transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [9]:
# store all the predictions of test data in a list
def return_predictons(model, test_loader):
    predictions = []
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            # move the images and labels to GPU
            images, labels = images.to(device), labels.to(device)
            outputs = model.forward(images)
            _, predicted = torch.max(outputs.data, 1)
            predictions.extend(predicted)
    return predictions

In [10]:
# store all the images names in a list for eg 1.jpeg, 2.jpeg, 3.jpeg

def return_images_names(test_data):
    images_names = []
    for i in range(len(test_data)):
        images_names.append(test_data.imgs[i][0].split('/')[-1])
    return images_names


In [11]:
# # write the images names and the predictions in a csv file with ID and Category as the column names
# import csv
# with open('resnet34_expoch.csv', mode='w') as file:
#     writer = csv.writer(file)
#     writer.writerow(['ID', 'Category'])
#     for i in range(len(images_names)):
#         writer.writerow([images_names[i], label_map[predictions[i].item()]])

In [12]:
def generate_csv_for_each_epoch(model, test_loader, test_data, epoch):
    predictions = return_predictons(model, test_loader)
    images_names = return_images_names(test_data)
    with open('./teja/resnet34_epoch'+str(epoch)+'.csv', mode='w') as file:
        writer = csv.writer(file)
        writer.writerow(['ID', 'Category'])
        for i in range(len(images_names)):
            writer.writerow([images_names[i], label_map[predictions[i].item()]])

In [13]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=2):
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_predictions = 0
        total_samples = 0
        
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)
        
        # Print the running loss and accuracy
        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = correct_predictions / total_samples
        print(f'Epoch {epoch+1}, Loss: {epoch_loss}, Accuracy: {epoch_accuracy}')
        print(f'Finished epoch {epoch}.')
        
        generate_csv_for_each_epoch(model, test_loader, test_data, epoch)
        print('CSV file generated for epoch', epoch)


In [14]:
def save_and_load_model_weights(model, filename):
    # Save model weights
    torch.save(model.state_dict(), filename)
    print(f"Model weights saved to '{filename}'")

    # Load model weights
    model.load_state_dict(torch.load(filename))
    print(f"Model weights loaded from '{filename}'")

In [15]:
train_model(model, train_loader, criterion, optimizer, num_epochs=15)
# save_and_load_model_weights(model, 'model_weights.pth')

Epoch 1, Loss: 3.334951640114071, Accuracy: 0.15096923076923077
Finished epoch 0.
CSV file generated for epoch 0
Epoch 2, Loss: 2.9096211796905114, Accuracy: 0.2366769230769231
Finished epoch 1.
CSV file generated for epoch 1
Epoch 3, Loss: 2.7025648049247546, Accuracy: 0.2828769230769231
Finished epoch 2.
CSV file generated for epoch 2
Epoch 4, Loss: 2.503279205030344, Accuracy: 0.32703076923076924
Finished epoch 3.
CSV file generated for epoch 3
Epoch 5, Loss: 2.28893754220619, Accuracy: 0.37901538461538464
Finished epoch 4.
CSV file generated for epoch 4
Epoch 6, Loss: 2.0739939156479723, Accuracy: 0.4276615384615385
Finished epoch 5.
CSV file generated for epoch 5
Epoch 7, Loss: 1.8419434500960854, Accuracy: 0.4816923076923077
Finished epoch 6.
CSV file generated for epoch 6
Epoch 8, Loss: 1.5626796161330592, Accuracy: 0.5475230769230769
Finished epoch 7.
CSV file generated for epoch 7
Epoch 9, Loss: 1.2259568725602599, Accuracy: 0.636676923076923
Finished epoch 8.
CSV file generat